<a id="sectionx"></a>
# MPO FD Rollover Testing - Notebook
* [Get FDs Maturing today](#section1)
* [Create new Rollover Term Deposit - Monthly (manual)](#section2)
* [Create new Rollover Term Deposit - Monthly (in one step)](#section2b)



GitHub: https://github.com/MkershMambu/MambuAPINotebook/blob/master/MPO%20FD%20Rollover%20Test.ipynb

Viewer: [https://nbviewer.jupyter.org/github/MkershMambu/MambuAPINotebook/blob/master/MPO%20FD%20Rollover%20Test.ipynb](https://nbviewer.jupyter.org/github/MkershMambu/MambuAPINotebook/blob/master/MPO%20FD%20Rollover%20Test.ipynb?flush_cache=true)



In [9]:
%run API.py
%run ENV.py
from IPython.core.display import HTML
setENV(ENV)

<a id="section1"></a>
[back](#sectionx)
### API call to determine FD's maturing today
* This was in the original spec doc but isn't the call that is being made
* For demo purposes though it may make sense to use this filter

In [50]:
%%writefile FDJSONBODY
{
  "filterConstraints": [
      {
          "filterSelection": "MATURITY_DATE",
          "filterElement": "TODAY"
      },
        {
            "filterElement": "IN",
            "filterSelection": "ACCOUNT_STATE",
            "values": [
                "ACTIVE"
            ]
        }
  ]
}

Writing FDJSONBODY


In [18]:
HEADERS = {'Content-Type': 'application/json'}
PARAMS = {}
r = POST('{{env1}}/savings/search', headers=HEADERS, params=PARAMS, body="FDJSONBODY")
print(len(r.json()))
PRINT(r)

1
API status: 200
JSON Response:
[
    {
        "accountHolderKey": "8a8186016a73c831016a788d7fa32d2e",
        "accountHolderType": "CLIENT",
        "accountState": "ACTIVE",
        "accountType": "FIXED_DEPOSIT",
        "accruedInterest": "0",
        "activationDate": "2018-11-06T00:00:00+0000",
        "allowOverdraft": false,
        "approvedDate": "2019-11-06T09:57:56+0000",
        "assignedBranchKey": "8a8186016a73c831016a77fe7fcf2b21",
        "availableBalance": "5571.59",
        "balance": "5571.59",
        "creationDate": "2019-11-06T08:57:55+0000",
        "currency": {
            "code": "EUR",
            "creationDate": "2018-12-05T14:25:48+0000",
            "currencySymbolPosition": "BEFORE_NUMBER",
            "digitsAfterDecimal": 2,
            "isBaseCurrency": true,
            "lastModifiedDate": "2018-12-05T14:25:48+0000",
            "name": "Euro",
            "symbol": "\u20ac"
        },
        "currencyCode": "EUR",
        "encodedKey": "8a81862f

### The API that the Rollover App was using
* NOTE: Need EOD to run before FD(s) moved into MATURED state
    * So not great for demos
* Best option in a live PROD enviornment though

In [52]:
%%writefile FDJSONBODY1
{
  "filterConstraints": [
      {
          "filterElement": "IN",
          "filterSelection": "ACCOUNT_STATE",
          "values": [
              "MATURED"
          ]
          
      }
  ]
}

Writing FDJSONBODY1


In [53]:
HEADERS = {'Content-Type': 'application/json'}
PARAMS = {}
r = POST('{{env1}}/savings/search', headers=HEADERS, params=PARAMS, body="FDJSONBODY1")
print(len(r.json()))
PRINT(r)

2
API status: 200
JSON Response:
[
    {
        "accountHolderKey": "8a8186ac692678910169288509c606bc",
        "accountHolderType": "CLIENT",
        "accountState": "MATURED",
        "accountType": "FIXED_DEPOSIT",
        "accruedInterest": "0",
        "activationDate": "2019-02-26T07:54:45+0000",
        "allowOverdraft": false,
        "approvedDate": "2019-02-26T07:54:34+0000",
        "assignedBranchKey": "8a8186ac692678910169287cf43606af",
        "availableBalance": "52727.4",
        "balance": "102727.4",
        "creationDate": "2019-02-26T06:54:24+0000",
        "currency": {
            "code": "EUR",
            "creationDate": "2018-12-05T14:25:48+0000",
            "currencySymbolPosition": "BEFORE_NUMBER",
            "digitsAfterDecimal": 2,
            "isBaseCurrency": true,
            "lastModifiedDate": "2018-12-05T14:25:48+0000",
            "name": "Euro",
            "symbol": "\u20ac"
        },
        "currencyCode": "EUR",
        "encodedKey": "8a8186

<a id="section2"></a>
[back](#sectionx)
### Create new Rollover Term Deposit - Monthly (manual)
* With maturity of today so that it triggers the Rollover App
* This section takes you through step-by-step
   * To perform via one function call see XXX

In [54]:
%%writefile FDJSONBODY2
{
  "accountType": "FIXED_DEPOSIT",
  "name": "FD Rollover - Monthly",
  "accountHolderKey": "8a8186016a73c831016a788d7fa32d2e",
  "productTypeKey": "8a8186016a73c831016a7807c0bb2b2a",
  "currencyCode": "EUR",
  "accountHolderType": "CLIENT",
  "interestSettings": {
      "interestRateSettings": {
            "encodedKey": "8a8186016a7a9872016a7e8d7e7e179e",
            "interestChargeFrequency": "ANNUALIZED",
            "interestChargeFrequencyCount": 1,
            "interestRate": 2.54,
            "interestRateTerms": "FIXED",
            "interestRateTiers": []
        }
    }
}


Writing FDJSONBODY2


In [55]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
r = POST('{{env1}}/deposits', headers=HEADERS, params=PARAMS, body="FDJSONBODY2")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)
encodedKey = r.json()["encodedKey"]

API status:
201
Now the JSON:
{"encodedKey":"8a81865e6e307e8e016e314a7aff027a","creationDate":"2019-11-03T13:46:23+01:00","lastModifiedDate":"2019-11-03T13:46:23+01:00","id":"NFJB374","name":"FD Rollover - Monthly","accountHolderType":"CLIENT","accountHolderKey":"8a8186016a73c831016a788d7fa32d2e","accountState":"PENDING_APPROVAL","productTypeKey":"8a8186016a73c831016a7807c0bb2b2a","accountType":"FIXED_DEPOSIT","currencyCode":"EUR","assignedBranchKey":"8a8186016a73c831016a77fe7fcf2b21","internalControls":{},"overdraftSettings":{"allowOverdraft":false,"overdraftLimit":0},"interestSettings":{"interestRateSettings":{"encodedKey":"8a81865e6e307e8e016e314d595e027b","interestRate":2.54,"interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED"},"interestPaymentSettings":{"interestPaymentPoint":"EVERY_MONTH","interestPaymentDates":[]}},"overdraftInterestSettings":{},"balances":{"totalBalance":0,"overdraftAmount":0,"technicalOverdr

### Approve the account

In [56]:
%%writefile FDJSONBODY3
{
  "action": "APPROVE",
  "notes": "more notes"
}

Writing FDJSONBODY3


In [57]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}:changeState".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="FDJSONBODY3")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
200
Now the JSON:
{"encodedKey":"8a81865e6e307e8e016e314a7aff027a","creationDate":"2019-11-03T13:46:23+01:00","lastModifiedDate":"2019-11-03T13:46:36+01:00","id":"NFJB374","name":"FD Rollover - Monthly","accountHolderType":"CLIENT","accountHolderKey":"8a8186016a73c831016a788d7fa32d2e","accountState":"APPROVED","productTypeKey":"8a8186016a73c831016a7807c0bb2b2a","accountType":"FIXED_DEPOSIT","approvedDate":"2019-11-03T13:46:36+01:00","currencyCode":"EUR","assignedBranchKey":"8a8186016a73c831016a77fe7fcf2b21","internalControls":{},"overdraftSettings":{"allowOverdraft":false,"overdraftLimit":0},"interestSettings":{"interestRateSettings":{"encodedKey":"8a81865e6e307e8e016e314d595e027b","interestRate":2.54000000000000000000,"interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED"},"interestPaymentSettings":{"interestPaymentPoint":"EVERY_MONTH","interestPaymentDates":[]}},"overdraftInterestSettings":{},"balances":{

### Deposit into account

In [16]:
%%writefile FDJSONBODY4
{
  "transactionDetails": {
    "transactionChannelId": "8a818e74677a2e9201677ec2b4c336a6"
  },
  "amount": 5432,
  "notes": "some notes",
  "paymentOrderId": "{{UniqueID}}",
  "externalId": "{{UniqueID}}",
  "valueDate": "2018-11-06T00:00:00+01:00"
}

Overwriting FDJSONBODY4


In [59]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
BODYPARTS = {'UniqueID': getUUID()}
url = "{{env1}}" + "/deposits/{0}/deposit-transactions".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="FDJSONBODY4", bodyparts=BODYPARTS)
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
201
Now the JSON:
{"encodedKey":"8a8187b26e30ebff016e314a81ae002c","id":"470","externalId":"8e5c5004-6423-4bd7-b8e0-859ce6a84a51","paymentOrderId":"8e5c5004-6423-4bd7-b8e0-859ce6a84a51","creationDate":"2019-11-03T13:46:53+01:00","valueDate":"2018-11-03T00:00:00+01:00","notes":"some notes","parentAccountKey":"8a81865e6e307e8e016e314a7aff027a","type":"DEPOSIT","amount":5432,"currencyCode":"EUR","affectedAmounts":{"fundsAmount":5432.0000000000,"interestAmount":0,"feesAmount":0,"overdraftAmount":0,"overdraftFeesAmount":0,"overdraftInterestAmount":0,"technicalOverdraftAmount":0,"technicalOverdraftInterestAmount":0,"fractionAmount":0},"taxes":{},"accountBalances":{"totalBalance":5432.0000000000},"userKey":"8a8186ac69267891016928db604e0735","branchKey":"8a8186016a73c831016a77fe7fcf2b21","terms":{"interestSettings":{"interestRate":2.54000000000000000000},"overdraftInterestSettings":{},"overdraftSettings":{}},"transactionDetails":{"transactionChannelKey":"8a818e74677a2e9201677ec2b4c

### Start the maturity

In [60]:
%%writefile FDJSONBODY5
{
  "type": "START_MATURITY",
  "notes": "start maturity for FD"
}

Writing FDJSONBODY5


In [61]:
HEADERS = {'Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/savings/{0}/transactions".format(encodedKey)

r = POST(url, headers=HEADERS, params=PARAMS, body="FDJSONBODY5")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
200
Now the JSON:
{"encodedKey":"8a81865e6e307e8e016e314a7aff027a","id":"NFJB374","accountHolderKey":"8a8186016a73c831016a788d7fa32d2e","accountHolderType":"CLIENT","name":"FD Rollover - Monthly","creationDate":"2019-11-03T12:46:23+0000","approvedDate":"2019-11-03T13:46:36+0000","activationDate":"2018-11-03T00:00:00+0000","lastModifiedDate":"2019-11-03T12:47:10+0000","lastInterestCalculationDate":"2019-11-03T00:00:00+0000","lastInterestStoredDate":"2019-11-03T00:00:00+0000","productTypeKey":"8a8186016a73c831016a7807c0bb2b2a","accountType":"FIXED_DEPOSIT","accountState":"ACTIVE","maturityDate":"2019-11-03T00:00:00+0000","balance":"5571.59","accruedInterest":"0","overdraftInterestAccrued":"0","technicalOverdraftInterestAccrued":"0","overdraftAmount":"0","technicalOverdraftAmount":"0","interestSettings":{"interestRate":"2.54","encodedKey":"8a81865e6e307e8e016e314d595e027b","interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateSource":"FIXED_INTER

<a id="section2b"></a>
[back](#sectionx)
### Create new Rollover Term Deposit - Monthly (in one step)
* This section will only be relevant if you have an executable version of this notebook

In [12]:
def createFDMonthly_MaturesToday():
    # Create a new FD
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env1}}/deposits', headers=HEADERS, params=PARAMS, body="FDJSONBODY2")
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)
    encodedKey = r.json()["encodedKey"]
    
    # Approve the account
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    url = "{{env1}}" + "/deposits/{0}:changeState".format(encodedKey)
    r = POST(url, headers=HEADERS, params=PARAMS, body="FDJSONBODY3")
    
    # Deposit into the account
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    BODYPARTS = {'UniqueID': getUUID()}
    url = "{{env1}}" + "/deposits/{0}/deposit-transactions".format(encodedKey)
    r = POST(url, headers=HEADERS, params=PARAMS, body="FDJSONBODY4", bodyparts=BODYPARTS)
    
    # Start the Maturity
    HEADERS = {'Content-Type': 'application/json'}
    PARAMS = {}
    url = "{{env1}}" + "/savings/{0}/transactions".format(encodedKey)

    r = POST(url, headers=HEADERS, params=PARAMS, body="FDJSONBODY5")

In [17]:
createFDMonthly_MaturesToday()

API status:
201
Now the JSON:
{"encodedKey":"8a81862f6e3f8152016e3fee48c200e7","creationDate":"2019-11-06T09:57:55+01:00","lastModifiedDate":"2019-11-06T09:57:55+01:00","id":"LZVP918","name":"FD Rollover - Monthly","accountHolderType":"CLIENT","accountHolderKey":"8a8186016a73c831016a788d7fa32d2e","accountState":"PENDING_APPROVAL","productTypeKey":"8a8186016a73c831016a7807c0bb2b2a","accountType":"FIXED_DEPOSIT","currencyCode":"EUR","assignedBranchKey":"8a8186016a73c831016a77fe7fcf2b21","internalControls":{},"overdraftSettings":{"allowOverdraft":false,"overdraftLimit":0},"interestSettings":{"interestRateSettings":{"encodedKey":"8a81862f6e3f8152016e3fee48c200e8","interestRate":2.54,"interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED"},"interestPaymentSettings":{"interestPaymentPoint":"EVERY_MONTH","interestPaymentDates":[]}},"overdraftInterestSettings":{},"balances":{"totalBalance":0,"overdraftAmount":0,"technicalOverdr

## Miscellaneous - Helper APIs

### Delete Account

In [48]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
r = DELETE(url, headers=HEADERS, params=PARAMS)
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
400
Now the JSON:
{"errors":[{"errorCode":105,"errorReason":"INVALID_ACCOUNT_STATE"}]}


### Get account details

In [49]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json'}
# Parameters below set up pagination and only returns max 20
PARAMS = {'detailsLevel': 'FULL', 'paginationDetails': 'ON','limit': 20}
r = GET('{{env1}}/deposits/APIS515', headers=HEADERS, params=PARAMS)
PRINT(r)

API status: 200
JSON Response:
{
    "accountHolderKey": "8a8186016a73c831016a788d7fa32d2e",
    "accountHolderType": "CLIENT",
    "accountState": "CLOSED",
    "accountType": "FIXED_DEPOSIT",
    "accruedAmounts": {
        "interestAccrued": 0,
        "overdraftInterestAccrued": 0,
        "technicalOverdraftInterestAccrued": 0
    },
    "activationDate": "2018-05-02T00:00:00+01:00",
    "approvedDate": "2019-05-03T16:30:36+01:00",
    "assignedBranchKey": "8a8186016a73c831016a77fe7fcf2b21",
    "balances": {
        "availableBalance": 0,
        "feesDue": 0,
        "holdBalance": 0,
        "lockedBalance": 0,
        "overdraftAmount": 0,
        "overdraftInterestDue": 0,
        "technicalOverdraftAmount": 0,
        "technicalOverdraftInterestDue": 0,
        "totalBalance": 0
    },
    "closedDate": "2019-05-04T11:55:52+01:00",
    "creationDate": "2019-05-03T15:30:21+01:00",
    "currencyCode": "EUR",
    "encodedKey": "8a8186016a7a9872016a7e19680c0ec5",
    "id": "APIS